<a href="https://colab.research.google.com/github/aniketvakil/Movie-Recommendation-System/blob/master/Movie_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install python-rake
!pip install rake-nltk


import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [0]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [0]:
def recommendations(Title,parameter):
  
  df['bag_of_words'] = df['Genre']+ ' ' + df['Director']+ ' ' +df[parameter]+ ' '+df[parameter]+ ' '+df[parameter]
  #df.head()
  n=len(df)
  for i in range(n):
    df['bag_of_words'][i]= df['bag_of_words'][i]+' ' +str(df['Key_words'][i]).strip('[]')
  #print(df['bag_of_words'][1])
  #df.drop(columns = ['Key_words'], inplace = True)
  #df.head()
  df.set_index('Title', inplace = True)
  df['bag_of_words']= df['bag_of_words'].str.replace("'","")
  df['bag_of_words']= df['bag_of_words'].str.replace(',','')
  df['bag_of_words']= df['bag_of_words'].str.replace('.','')
  count = CountVectorizer()
  count_matrix = count.fit_transform(df['bag_of_words'])
  indices = pd.Series(df.index)
  cosine_sim = cosine_similarity(count_matrix, count_matrix)
  recommended_movies = []
  # gettin the index of the movie that matches the title
  idx = indices[indices == Title].index[0]
  # creating a Series with the similarity scores in descending order
  score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
  # getting the indexes of the 10 most similar movies
  top_10_indexes = list(score_series.iloc[1:11].index)
  # populating the list with the titles of the best 10 matching movies
  for i in top_10_indexes:
    recommended_movies.append(list(df.index)[i])
        
  return recommended_movies
  

In [0]:
recommendations('Fargo','Actors')

['The Big Lebowski',
 'Network',
 'Monsters, Inc.',
 'The Shawshank Redemption',
 'Papillon',
 'JFK',
 'Harvey',
 'The Graduate',
 "One Flew Over the Cuckoo's Nest",
 'Lawrence of Arabia']

In [0]:
recommendations('Fargo','Actors')

['The Big Lebowski',
 'Network',
 'Monsters, Inc.',
 'The Shawshank Redemption',
 'Papillon',
 'JFK',
 'Harvey',
 'The Graduate',
 "One Flew Over the Cuckoo's Nest",
 'Lawrence of Arabia']